In [16]:
import os, math, numpy as np
import pandas as pd
import copy
from collections import defaultdict
from docplex.mp.model import Model
import matplotlib.pyplot as plt

In [17]:
### Calculate travel time between two points: origin and destination
def travel_time(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

#### Function that reads in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [18]:
# Read instance information
def read_instance_information2(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    locations.set_index('id',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations, meters_per_minute, pickup_service_minutes, dropoff_service_minutes, \
            target_click_to_door, pay_per_order,\
            guaranteed_pay_per_hour

# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon and to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [19]:
# Accorting to default values in the paper
f_minute = 5
delta_u = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned
X = 25 # max pickup delay for commitment strategy, should be tuned
omega = 1000 # control undelivered orders
commitment_strategy = 0 # 0: no commitment, 1: commitment

# Defining objects

**Order Object**

In [20]:
class Order:
    def __init__(self, order_information : dict):

        self.id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')
        
        # To be updated after assignment
        self.pickup_time = 0
        self.dropoff_time = 0
        self.courier_id = ""
        self.assign_time = 0

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [21]:
class Route(object):
    def __init__(self,bundle : list, restaurant_id : str): 
        self.bundle = bundle 
        self.restaurant_id = restaurant_id
        
    def get_ready_time(self):
        ready_time = max([o.ready_time for o in self.bundle])
        return ready_time

    # calculate total travel time from 1st destination to the last destination of the route
    # do not include pickup service time and drop off service time
    def get_total_travel_time(self,meters_per_minute,locations):
        travel_points = [self.restaurant_id]+ [o.id for o in self.bundle]
        if len(travel_points) == 1:
            return 0
        else:
            total_travel_time = 0
            for i in range(len(travel_points)-1):
                total_travel_time += travel_time(travel_points[i], travel_points[i+1],meters_per_minute,locations)
            return total_travel_time

    def get_end_position(self,meters_per_minute,locations):
        return self.bundle[-1].id

    # calculate total service delay 
    # service delay = arrival_time at customer place - ready time (ignoring pickup service time and dropoff service time)
    def get_total_service_delay(self,meters_per_minute,locations):
        travel_points = [self.restaurant_id]+ [o.id for o in self.bundle]
        if len(travel_points) == 1:
            return 0
        else:
            total_service_delay = 0
            arrival_time_at_cp = self.get_ready_time()
            for i in range(len(travel_points)-1):
                arrival_time_at_cp += (travel_time(travel_points[i], travel_points[i+1],meters_per_minute,locations))
                total_service_delay += (arrival_time_at_cp - self.bundle[i].ready_time)
            return total_service_delay

    # calculate total_click_to_door 
    # click_to_door = arrival_time at customer place - placement time (ignoring pickup service time and dropoff service time)
    # the below function implement 
    def get_total_service_waiting(self,meters_per_minute,locations):
        travel_points = [self.restaurant_id]+[o.id for o in self.bundle]
        if len(travel_points) == 1:
            return 0
        else:
            total_service_waiting = 0
            arrival_time_at_cp = self.get_ready_time()
            for i in range(len(travel_points)-1):
                arrival_time_at_cp += (travel_time(travel_points[i], travel_points[i+1],meters_per_minute,locations))
                total_service_waiting += (arrival_time_at_cp - self.bundle[i].placement_time)
            return total_service_waiting

    # calculate route efficiency: travel time per order:
    def route_efficiency(self,meters_per_minute,locations):
        return len(self.bundle) / self.get_total_travel_time(meters_per_minute,locations)

    # calculate route cost
    def get_route_cost(self,meters_per_minute,locations):
        route_cost = self.get_total_travel_time(meters_per_minute,locations) + beta * self.get_total_service_delay(meters_per_minute,locations) + gamma * self.get_total_service_waiting(meters_per_minute,locations)
        return route_cost


**Assignment Object**

In [22]:
class Assignment():
    def __init__(self, assign_time : int , restaurant_id : str, courier: object, route : Route):
        self.assign_time = assign_time
        self.restaurant_id = restaurant_id  # each assignment of a bundle has only one corresponding restaurant
        self.pickup_time = 0
        self.courier = courier
        self.route = route
        self.isfinal_flag = 0 # indicate if the assignment is final (can not be updated)
        self.update_time = 0
        self.departure_time = 0         # the departure time of the courier to the restaurant of the assignment
        self.departure_location = '' # the last position of the courier before the assignment

    def update(self, new_assignment, meters_per_minute,locations):
        # update the new assignment into the old assignment
        # combine orders in the new assigment with orders in the old assignment
        for o in new_assignment.route.bundle:
            n = len(self.route.bundle)
            min_route_cost = float('inf')
            best_pos = 0
            for pos in range(n+1):
                self.route.bundle.insert(pos,o)
                route_cost = self.route.get_route_cost(meters_per_minute,locations)
                if route_cost < min_route_cost:
                    min_route_cost = route_cost
                    best_pos = pos
                self.route.bundle.pop(pos)
            self.route.bundle.insert(best_pos, o)
        # if the new assigment is final (isfinal_flage = 1), set isfinal_flag of the combination as 1
        if new_assignment.isfinal_flag == 1:
            self.isfinal_flag = 1
        else: # if the new assigment is not final (isfinal_flage = 0)
            # if there is an order that has been ready for x minutes, set assignment as final
            if not self.is_no_order_long_ready_time():
                self.isfinal_flag = 1
            else: # if there is no order that has been ready for x minutes, set assignment as tentative
                self.isfinal_flag = 0
        self.update_time +=1
        self.assign_time = new_assignment.assign_time

    def is_no_order_long_ready_time(self, x = X) -> bool:
        for o in self.route.bundle:
            route_ready_time = self.route.get_ready_time()
            if route_ready_time - o.ready_time >= x:
                return False
        return True

**Courier Object**

In [23]:
class Courier(object):
    def __init__(self, courier_information : dict, ):
        self.id = courier_information.get('courier')
        self.x = courier_information.get('x')
        self.y = courier_information.get('y')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Sequence of moves of the courier
        # Update and derived along the way
        self.assignments = []                           # containing assigned assigments 
        self.next_available_time = self.on_time                # when the courier is available for the next assignment (or the drop off time of the last order of the last assignment)
        self.position_after_last_assignment = self.id  # the position of the courier after completing the last assignment     
             

**DeliveryRouting Object**

In [25]:
class DeliveryRouting:
    def __init__(self, instance_dir : str):

        orders, restaurants, couriers, instanceparams, locations ,\
        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = read_instance_information2(instance_dir)


        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.id)

        self.restaurants = restaurants
        self.couriers = [Courier(courier) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = self.copy(self.orders)
        
        self.orders_by_horizon_interval = defaultdict(list)
        self.locations = locations



    def travel_time(self, origin_id,destination_id):
        dist=np.sqrt((self.locations.at[destination_id,'x']-self.locations.at[origin_id,'x'])**2\
                    +(self.locations.at[destination_id,'y']-self.locations.at[origin_id,'y'])**2)
        tt=np.ceil(dist/self.meters_per_minute)
        return tt

    def copy(self,x):
        return copy.deepcopy(x)

    def get_ready_orders(self) -> dict:
        '''
        This function return orders which have ready time fall into the corresponding horizon.
        This function should be run only once.
        '''
        # starting time of each interval
        t_list = [*range(0,24*60+1,f_minute)]
        # get ready orders correspoding to each horizon interval
        for i in range(1,len(t_list)):
            for o in self.orders:
                if o.placement_time < t_list[i] and o.placement_time >= t_list[i-1] and o.ready_time < t_list[i]+delta_u:
                    self.orders_by_horizon_interval[t_list[i]].append(o)
                if o.placement_time < t_list[i] and o.placement_time >= t_list[i-1] and o.ready_time >= t_list[i]+delta_u:
                    self.orders_by_horizon_interval[t_list[i]+f_minute*np.ceil((o.ready_time - (t_list[i]+delta_u))/f_minute)].append(o)

    def get_ready_orders_at_t(self,t):
        return self.orders_by_horizon_interval[t]

    def get_idle_courier_at_t(self,t):
        idle_courier = []
        for c in self.couriers:
            if c.next_available_time < t+delta_u and c.next_available_time < c.off_time :
                idle_courier.append(c)
        return idle_courier

    def get_bundle_size(self, t) -> int :
        
        number_of_orders = len(self.get_ready_orders_at_t(t))
        number_of_couriers = len(self.get_idle_courier_at_t(t))
        if number_of_couriers == 0:
            bundle_size = 2 # default value of bundle size
        else:
            bundle_size = np.ceil(number_of_orders/number_of_couriers)
        return bundle_size

    # check if the courier can take a bundle. 
    def can_assign(self, t, courier, route : Route) -> bool:
        route_ready_time = route.get_ready_time()
        if route_ready_time < courier.on_time:
            return False
        if route_ready_time> courier.off_time:
            return False
        arrival_time = max(t,courier.next_available_time) +\
                        self.dropoff_service_minutes/2 +\
                         self.travel_time(courier.position_after_last_assignment,route.restaurant_id) +\
                          self.pickup_service_minutes/2
        if arrival_time > courier.off_time:
            return False
        if len(courier.assignments) > 0:
            if courier.assignments[-1].isfinal_flag == 0:
                if courier.assignments[-1].restaurant_id != route.restaurant_id:
                    return False
        return True

    # assign a bundle to a courier
    def assign_bundle(self, t: int, courier: Courier, route: Route):
        # calculate courier's arrival time to the bundle's restaurant:
        arrival_time = max(t,courier.next_available_time) +\
                        self.dropoff_service_minutes/2 +\
                         self.travel_time(courier.position_after_last_assignment,route.restaurant_id) +\
                          self.pickup_service_minutes/2
        route_ready_time = route.get_ready_time()
        pickup_time = max(arrival_time,route_ready_time)
        assignment = Assignment(t, route.restaurant_id, courier, route)
        # update assignment attributes:
        assignment.pickup_time = pickup_time
        assignment.departure_time = max(t,courier.next_available_time) +self.dropoff_service_minutes/2 
        assignment.departure_location = courier.position_after_last_assignment
        # update order attributes:
        for i, order in enumerate(route.bundle):
            order.pickup_time = pickup_time
            order.assign_time = t
            order.courier_id = courier.id
            if i == 0:
                order.dropoff_time = pickup_time + self.pickup_service_minutes/2 +\
                                        self.travel_time(route.restaurant_id,order.id) +\
                                        self.dropoff_service_minutes/2 
            else:
                order.dropoff_time = route.bundle[i-1].dropoff_time +\
                                        self.dropoff_service_minutes/2 +\
                                        self.travel_time(route.bundle[i-1].id,order.id) +\
                                        self.dropoff_service_minutes/2

        ### Commitment strategy
        # If d can reach restaurant r before t + f and all orders in s are estimated to be ready by t + f,
        # make a final commitment of d to s: instruct d to travel to rs, pick up and deliver orders in s.
        if (arrival_time <= t+f_minute and route_ready_time <= t+f_minute) or\
           (route_ready_time <= t+f_minute and route_ready_time <= arrival_time):
            assignment.isfinal_flag = 1
            
            if len(courier.assignments) > 0:
                # if the last assignment can be updated
                # the last assignment can be updated if its isfinal_flag = 0
                if courier.assignments[-1].isfinal_flag == 0:
                    # Combine bundle
                    courier.assignments[-1].update(assignment,self.meters_per_minute,self.locations) # the old courier.assignments[-1] is combined with the new assignment to become new courier.assignments[-1]
                    courier.assignments[-1].pickup_time = max(arrival_time,courier.assignments[-1].route.get_ready_time())
                    courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                    courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2     
                    courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)
                    # update order attributes:
                    for i, order in enumerate(courier.assignments[-1].route.bundle):
                        order.pickup_time = pickup_time
                        order.courier_id = courier.id
                        if i == 0:
                            order.dropoff_time = pickup_time + self.pickup_service_minutes/2 +\
                                                    self.travel_time(courier.assignments[-1].restaurant_id,order.id) +\
                                                    self.dropoff_service_minutes/2 
                        else:
                            order.dropoff_time = courier.assignments[-1].route.bundle[i-1].dropoff_time +\
                                                    self.dropoff_service_minutes/2 +\
                                                    self.travel_time(courier.assignments[-1].route.bundle[i-1].id,order.id) +\
                                                    self.dropoff_service_minutes/2
                else: # if the last assignment can not be updated
                    courier.assignments.append(assignment)
                    courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                        courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2
                    courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)
            else: 
                courier.assignments.append(assignment)
                courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                        courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2
                courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)
        
        else:
            if commitment_strategy == 0:
                assignment.isfinal_flag = 1
            else:
                assignment.isfinal_flag = 0
            
            if len(courier.assignments) > 0:
                # if the last assignment can be updated
                # the last assignment can be updated if its isfinal_flag = 0
                if courier.assignments[-1].isfinal_flag == 0:
                    # Combine bundle
                    courier.assignments[-1].update(assignment,self.meters_per_minute,self.locations) # the old courier.assignments[-1] is combined with the new assignment to become new courier.assignments[-1]
                    courier.assignments[-1].pickup_time = max(arrival_time,courier.assignments[-1].route.get_ready_time())
                    if courier.assignments[-1].isfinal_flag == 1:
                        courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                        courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2
                        courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)
                    else:
                        pass
                    # update order attributes:
                    for i, order in enumerate(courier.assignments[-1].route.bundle):
                        order.pickup_time = pickup_time
                        order.courier_id = courier.id
                        if i == 0:
                            order.dropoff_time = pickup_time + self.pickup_service_minutes/2 +\
                                                    self.travel_time(courier.assignments[-1].restaurant_id,order.id) +\
                                                    self.dropoff_service_minutes/2 
                        else:
                            order.dropoff_time = courier.assignments[-1].route.bundle[i-1].dropoff_time +\
                                                    self.dropoff_service_minutes/2 +\
                                                    self.travel_time(courier.assignments[-1].route.bundle[i-1].id,order.id) +\
                                                    self.dropoff_service_minutes/2
                else: # if the last assignment can not be updated
                    courier.assignments.append(assignment)
                    if courier.assignments[-1].isfinal_flag == 1:
                        courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                        courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2
                        courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)
            else: 
                courier.assignments.append(assignment)
                if courier.assignments[-1].isfinal_flag == 1:
                    courier.next_available_time = courier.assignments[-1].pickup_time +  self.pickup_service_minutes/2 +\
                                                        courier.assignments[-1].route.get_total_travel_time(self.meters_per_minute,self.locations) +\
                                                     self.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                      self.dropoff_service_minutes/2
                    courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(self.meters_per_minute,self.locations)

    def initialization(self, t:int, ready_orders: list, idle_couriers: list, bundle_size: int):
        list_of_routes_by_restaurant = []
        if not ready_orders:
            # print('b1')
            return  list_of_routes_by_restaurant
        else:
            # print('b2')
            for r_id in self.restaurants['restaurant']:
                # print(r_id)
                
                # build set of ready orders from restaurant r
                r_order= []
                for o in ready_orders:
                    if o.restaurant_id == r_id:
                        r_order.append(o)
                # print(r_order)
                # get number of bundel for restaurant r
                number_of_bundle = int(np.ceil(len(r_order)/bundle_size))
                # Initiate emptly lists
                set_of_bundles = [[] for _ in range(number_of_bundle)]
                
                # Assign orders into bundels:
                for o in r_order:
                    min_cost_increase = float('inf')
                    for i in range(number_of_bundle):
                        n = len(set_of_bundles[i])
                        if n + 1<= bundle_size:
                            #print('x')
                            min_route_cost = float('inf')
                            for pos in range(n+1):
                                set_of_bundles[i].insert(pos,o)
                                if Route(set_of_bundles[i], r_id).get_route_cost(self.meters_per_minute,self.locations) < min_route_cost:
                                    min_route_cost = Route(set_of_bundles[i], r_id).get_route_cost(self.meters_per_minute,self.locations)
                                    best_pos = pos
                                set_of_bundles[i].pop(pos)
                        else: # if existing size + 1 > bundle size and insertion decreases route efficiency then
                              # Disregard s for order o and finnd the next best route and insertion position
                            #print('o')
                            min_route_cost = float('inf')
                            for pos in range(n+1):
                                set_of_bundles[i].insert(pos,o)
                                if Route(set_of_bundles[i], r_id).get_route_cost(self.meters_per_minute,self.locations) < min_route_cost:
                                    min_route_cost = Route(set_of_bundles[i], r_id).get_route_cost(self.meters_per_minute,self.locations)
                                    best_pos = pos
                                set_of_bundles[i].pop(pos)
                            # get current efficiency
                            current_efficiency = n / Route(set_of_bundles[i],r_id).get_total_travel_time(self.meters_per_minute,self.locations)
                            # get new efficiency
                            set_of_bundles[i].insert(best_pos,o)
                            new_efficiency = (n+1) / Route(set_of_bundles[i],r_id).get_total_travel_time(self.meters_per_minute,self.locations)
                            if current_efficiency < new_efficiency:
                                set_of_bundles[i].pop(best_pos)
                            else: 
                                set_of_bundles[i].pop(best_pos)
                                continue 

                        current_cost = Route(set_of_bundles[i],r_id).get_route_cost(self.meters_per_minute,self.locations)
                        set_of_bundles[i].insert(best_pos, o)
                        new_cost = Route(set_of_bundles[i],r_id).get_route_cost(self.meters_per_minute,self.locations)
                        cost_increase = new_cost - current_cost
                        if cost_increase < min_cost_increase:
                            min_cost_increase = cost_increase
                            best_i = i
                            best_i_pos = best_pos
                        set_of_bundles[i].pop(best_pos)

                    # Assign o the best bundle and the best position within that bundle
                    set_of_bundles[best_i].insert(best_i_pos,o)
                
                set_of_bundles = [Route(bundle,r_id) for bundle in set_of_bundles]
                if set_of_bundles:
                    list_of_routes_by_restaurant.append(set_of_bundles)

            return list_of_routes_by_restaurant 
    

    #### Local Search ####
    def get_restaurant_cost(self, res:list): 
        res_cost = 0
        for route in res:
            res_cost += route.get_route_cost(self.meters_per_minute,self.locations)
        return res_cost
    
    def get_total_restaurant_cost(self, list_of_routes_by_restaurant:list): 
        total_res_cost = 0
        for res in list_of_routes_by_restaurant:
            for route in res:
                total_res_cost += route.get_route_cost(self.meters_per_minute,self.locations)
        return total_res_cost

    def local_search(self, list_of_routes_by_restaurant):
        # get current total cost: 
        current_total_res_cost = self.get_total_restaurant_cost(list_of_routes_by_restaurant)

        # perform local search
        for res in list_of_routes_by_restaurant:
            current_cost = self.get_restaurant_cost(res)
            for route1 in res:
                route1_copy = [o for o in route1.bundle]
                for o in route1_copy:
                    best_route = route1
                    best_pos = route1.bundle.index(o)
                    # remove order o from route1
                    route1.bundle.remove(o)
                    # find the best route and position to reinsert order o
                    for route2 in res:
                        for j in range(len(route2.bundle)+1):
                            # insert order at position j
                            route2.bundle.insert(j,o)
                            new_cost = self.get_restaurant_cost(res)
                            # if new cost is less than current cost then record the new position
                            if new_cost < current_cost:
                                current_cost = new_cost
                                best_route = route2
                                best_pos = j

                            route2.bundle.pop(j)
                    # insert order at best position
                    best_route.bundle.insert(best_pos,o)
        # delete route has no orders
        for res_index in range(len(list_of_routes_by_restaurant)):
            list_of_routes_by_restaurant[res_index] = [route for route in list_of_routes_by_restaurant[res_index] if  len(route.bundle)!= 0]
             
        # get new total cost
        new_total_res_cost = self.get_total_restaurant_cost(list_of_routes_by_restaurant)
        # if new_total_res_cost != current_total_res_cost:
        #     print('Old cost: ', current_total_res_cost)
        #     print('New cost: ', new_total_res_cost)
        return list_of_routes_by_restaurant

# Implement Algorithm

In [26]:
# Select data set
instance_dir = './data/5o50t75s1p100'
# Initiate delivery routing

In [27]:
# Running the algo
dr = DeliveryRouting(instance_dir)
dr.get_ready_orders()
def main_algo(dr: DeliveryRouting):
    # time horizon: running the algorithm every f minutes
    t_list = [*range(0, 24*60+1, f_minute)]
    # 
    for t in t_list:
        ready_orders = dr.get_ready_orders_at_t(t)
        # check if any order in the last bundle is late than X minutes
        # commit it as final
        for courier in dr.couriers:
            if len(courier.assignments) > 0:
                if courier.assignments[-1].isfinal_flag == 0:
                    for o in courier.assignments[-1].route.bundle:
                        if t - o.ready_time > X:
                            courier.assignments[-1].isfinal_flag = 1
                            # update courier attributes and order attributes
                            courier.next_available_time = courier.assignments[-1].pickup_time +  dr.pickup_service_minutes/2 +\
                                                            courier.assignments[-1].route.get_total_travel_time(dr.meters_per_minute,dr.locations) +\
                                                        dr.dropoff_service_minutes*(len(courier.assignments[-1].route.bundle)-1) +\
                                                        dr.dropoff_service_minutes/2
                            courier.position_after_last_assignment = courier.assignments[-1].route.get_end_position(dr.meters_per_minute,dr.locations)
                            break

        # get idle couriers
        idle_couriers = dr.get_idle_courier_at_t(t)
        bundle_size = int(dr.get_bundle_size(t))
        # if there are orders to be delivered, run the algorithm
        if len(ready_orders)>0:
            list_of_routes_by_restaurant = dr.initialization(t,ready_orders,idle_couriers,bundle_size)
            list_of_routes_by_restaurant = dr.local_search(list_of_routes_by_restaurant)

            # create mp model
            m = Model('bundle_assignment')

            list_of_route = [route for r in list_of_routes_by_restaurant for route in r]
            route_index = [i for i in range(len(list_of_route))]
            courier_index = [0]+[j+1 for j in range(len(idle_couriers))]
            route_courier_list = [(i,j) for i in route_index for j in courier_index]
            
            # create variables
            route_courier = m.binary_var_dict(route_courier_list, name='route_courier')
            can_assign = {}
            for i in route_index:
                for j in courier_index: 
                    if j>0:
                        if not dr.can_assign(t, idle_couriers[j-1],list_of_route[i]):
                            can_assign[(i,j)] = 0
                        else:
                            can_assign[(i,j)] = 1
            
            # set objective
            number_of_order_assign_to_pseudo_courier = m.sum(route_courier[i,0] for i in route_index)
            real_pickup_delay = 0
            for i in route_index:
                for j in courier_index:
                    if j>0:
                        route = list_of_route[i]
                        courier = idle_couriers[j-1]
                        arrival_time = max(t,courier.next_available_time) +\
                            dr.dropoff_service_minutes/2 +\
                            dr.travel_time(courier.position_after_last_assignment,route.restaurant_id) +\
                            dr.pickup_service_minutes/2
                        route_ready_time = route.get_ready_time()
                        real_pickup_delay += max(0,arrival_time-route_ready_time)*route_courier[i,j]
            m.minimize(omega*number_of_order_assign_to_pseudo_courier+real_pickup_delay)
            # constraints
            # each route is assigned to one courier
            for i in route_index:
                m.add_constraint(m.sum(route_courier[i,j] for j in courier_index)==1)
            # each courier is assigned to at most one route
            for j in courier_index:
                if j != 0:
                    m.add_constraint(m.sum(route_courier[i,j] for i in route_index)<=1)
            # check condition can_assign
            for i in route_index:
                for j in courier_index: 
                    if j>0:
                        m.add_constraint(route_courier[i,j]<=can_assign[(i,j)])
            
            # solve model
            solution = m.solve(log_output = False) 

            # assign routes to couriers
            for i in route_index:
                for j in courier_index:
                    if solution.get_value(route_courier[i,j]) == 1:
                        if j != 0:
                            dr.assign_bundle(t, idle_couriers[j-1], list_of_route[i])
    # Saving the final results
    # Saving order table
    order_table = pd.DataFrame(columns=['Order ID', 'Placement time', 'Ready time', 'Pickup time', 'Dropoff time', 'Courier ID'])
    for i, o in enumerate(dr.orders):
        order_table.loc[i,'Order ID'] = o.id
        order_table.loc[i,'Placement time'] = o.placement_time
        order_table.loc[i,'Ready time'] = o.ready_time
        order_table.loc[i,'Pickup time'] = o.pickup_time
        order_table.loc[i,'Dropoff time'] = o.dropoff_time
        order_table.loc[i,'Courier ID'] = o.courier_id
    
    df = order_table[order_table['Courier ID'] != ''] # do not include undelivered orders
    df.to_csv(os.path.join(instance_dir,'orders_solution_info.txt'), header=['order','placement_time','ready_time','pickup_time','dropoff_time','courier'], index=None, sep=' ', mode='w')

    # Saving courier table
    courier_table = pd.DataFrame(columns=['Courier ID', 'Departure time', 'Origin', 'Destination','Assignment ID'])
    assignment_id = 0
    for i, c in enumerate(dr.couriers):
        for a in c.assignments:
            assignment_id += 1
            # journey from courier's location/last assignment's last location to the restaurant of the first assignment
            courier_table = pd.concat([courier_table, pd.DataFrame(columns=['Courier ID', 'Departure time', 'Origin', 'Destination','Assignment ID'],
                                                data=[[c.id, a.departure_time, a.departure_location, a.restaurant_id, assignment_id]] )])
            
            # journey from restaurant to the last order's location
            # create departure_time
            departure_time = [a.pickup_time+dr.pickup_service_minutes/2]
            order_list = [a.restaurant_id]+[o.id for o in a.route.bundle]
            for j in range(1, len(order_list)-1):
                departure_time.append(departure_time[j-1]+dr.travel_time(order_list[j-1],order_list[j])+dr.dropoff_service_minutes)
            # create origin list
            origin_list = order_list[:len(order_list)-1]
            # create destination list
            destination_list = order_list[1:]
            # create table
            courier_table = pd.concat([courier_table, pd.DataFrame(columns=['Courier ID', 'Departure time', 'Origin', 'Destination','Assignment ID'],
                                                data=[[c.id, departure_time[j], origin_list[j], destination_list[j], assignment_id] for j in range(len(departure_time))])])
                    
    courier_table.to_csv(os.path.join(instance_dir,'courier_solution_info.txt'),\
                        header=None,\
                        index=None, sep=' ', mode='w')

    # Saving the assignment table
    assignment_table = pd.DataFrame(columns=['Assignment time', 'Pickup time', 'Courier ID', 'Orders','Assignment ID'])
    assignment_id = 0
    for i, c in enumerate(dr.couriers):
        for a in c.assignments:
            assignment_id += 1
            assignment_table = pd.concat([assignment_table, pd.DataFrame(columns=['Assignment time', 'Pickup time', 'Courier ID', 'Orders','Assignment ID'],
                                        data=[[a.assign_time, a.pickup_time, a.courier.id, [o.id for o in a.route.bundle],assignment_id]])])

    df = assignment_table[['Assignment time', 'Pickup time', 'Courier ID', 'Orders']].reset_index(drop=True)
    with open(os.path.join(instance_dir,'assignment_solution_info.txt'), 'w') as f:
        f.write('Assignment_time Pickup_time Courier_ID Orders\n')
    with open(os.path.join(instance_dir,'assignment_solution_info.txt'), 'a') as f: 
        for i in range(len(df)):
            values_list=[df['Assignment time'][i], df['Pickup time'][i], df['Courier ID'][i], ' '.join(df['Orders'][i])]
            values = ' '.join(map(str, values_list))+'\n'
            f.write(values)
        f.close
    return courier_table, order_table, assignment_table
courier_table, order_table, assignment_table = main_algo(dr)


In [30]:
def generate_courier_locations_by_timepoints(courier_table, dr):
    df_courier_expand = pd.DataFrame(columns=['Courier ID', 'time', 'x', 'y']).set_index('Courier ID')

    df_courier = courier_table.set_index('Courier ID').drop('Assignment ID', axis=1)
    courier_in_table = set(df_courier.index)
    for c in dr.couriers:
        c_id = c.id
        if c_id in courier_in_table:
            df = df_courier[df_courier.index == c_id]
        
            # add courier initial location
            df = pd.concat([df, pd.DataFrame(index=[c_id],columns=['Departure time', 'Origin', 'Destination'],
                                            data=[[c.on_time, c_id, c_id]])])
            # sort by departure time
            df = df.sort_values(by=['Departure time'])  
            # add last location
            arrive_time = max(df['Departure time']) + dr.travel_time(df['Origin'][-1],df['Destination'][-1]) +dr.dropoff_service_minutes
            df = pd.concat([df, pd.DataFrame(index=[c_id],columns=['Departure time', 'Origin', 'Destination'],
                                                data=[[arrive_time, df['Destination'][-1], df['Destination'][-1]]])])
            # add courier final location
            if max(df['Departure time']) < c.off_time:
                df = pd.concat([df, pd.DataFrame(index=[c_id],columns=['Departure time', 'Origin', 'Destination'],
                                                data=[[c.off_time, df['Destination'][-1], df['Destination'][-1]]])]) 
            else:
                pass
                
        else:
            df = pd.DataFrame(index=[c_id],columns=['Departure time', 'Origin', 'Destination'],
                                            data=[[c.on_time, c_id, c_id]])
            df = pd.concat([df, pd.DataFrame(index=[c_id],columns=['Departure time', 'Origin', 'Destination'],
                                            data=[[c.off_time, c_id, c_id]])])
        df['x1'], df['y1']=0, 0
        df.loc[c_id,'x1'] = df.loc[c_id,'Origin'].apply(lambda x: dr.locations.loc[x,'x'])
        df.loc[c_id,'y1'] = df.loc[c_id,'Origin'].apply(lambda x: dr.locations.loc[x,'y'])
        df['x2'], df['y2']=0, 0
        df.loc[c_id,'x2'] = df.loc[c_id,'Destination'].apply(lambda x: dr.locations.loc[x,'x'])
        df.loc[c_id,'y2'] = df.loc[c_id,'Destination'].apply(lambda x: dr.locations.loc[x,'y'])

        df = df.sort_values(by=['Departure time'])    
        # reset index
        df.reset_index(inplace=True)
        # iterate through the rows
        for i in range(len(df)-1):
            

            x1, y1, x2, y2 = df.loc[i,'x1'], df.loc[i,'y1'], df.loc[i,'x2'], df.loc[i,'y2']

            df_courier_expand = pd.concat([df_courier_expand, pd.DataFrame(columns=['time', 'x', 'y'],
                                                                        data = [[df.loc[i,'Departure time'],x1,y1]],
                                                                        index = [c_id])])
            # calculate the distance between the current row and the next row
            distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
            # calculate the time between the current row and the next row
            time_duration = df.loc[i+1,'Departure time'] - df.loc[i,'Departure time']
            # Generate locations for each time step
            locations = []
            for j in range(1,int(time_duration)):
                # Calculate the x and y coordinates for the current time step
                x = x1 + (x2 - x1) * (j / time_duration)
                y = y1 + (y2 - y1) * (j / time_duration)
                
                df_courier_expand = pd.concat([df_courier_expand, pd.DataFrame(columns=['time', 'x', 'y'],
                                                                        data = [[df.loc[i,'Departure time']+j,x,y]],
                                                                        index = [c_id])])
        df_courier_expand = pd.concat([df_courier_expand, pd.DataFrame(columns=['time', 'x', 'y'],
                                                                        data = [[df.loc[len(df)-1,'Departure time'],df.loc[len(df)-1,'x1'],df.loc[len(df)-1,'y1']]],
                                                                        index = [c_id])])

    return df_courier_expand

df_courier_expand = generate_courier_locations_by_timepoints(courier_table, dr)
df_courier_expand.to_csv(os.path.join(instance_dir,'df_courier_expand.txt'),\
                        header=None,\
                        index=True, sep=' ', mode='w')

In [13]:
df_courier_expand.head(191)

,time,x,y
c1,0.0,17260,19298
c1,1.0,17260.0,19298.0
c1,2.0,17260.0,19298.0
c1,3.0,17260.0,19298.0
c1,4.0,17260.0,19298.0
...,...,...,...
c2,1.0,18232.0,17854.0
c2,2.0,18232.0,17854.0
c2,3.0,18232.0,17854.0
c2,4.0,18232.0,17854.0


# Computing performance

In [14]:
from __future__ import print_function
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import os
import numpy as np
import bisect
import sys
'''
This script takes as input (at most) three directories, in the following order:
    1. instance directory: it is expected to contain files orders.txt, couriers.txt, restaurants.txt, and instance_parameters.txt
    2. input (solution) directory: it is expected to contain files assignment_solution_info.txt, orders_solution_info.txt, couriers_solution_info.txt
    3. output (summary) directory: the place where the output files will be stored
The script produces two files, named 'feasibility_check.txt' and 'solution_performance.txt'.
Example call:
    python compute_performance_summary.py instance_dir=instances/an_instance input_dir=solutions/my_instance/my_algorithm output_dir=performance_summaries/an_instance/my_algorithm
'''

# default directory
folder_default=os.path.join(os.path.curdir,'test_sampling_orders')

# some methods defined on their own for clarity
def travel_time(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

def parse_console_input_and_define_parameter_values(console_input):
    instance_dir=next((p for p in console_input if 'instance_dir=' in p),None)
    if instance_dir:
        # (containing orders.txt, couriers.txt, restaurants.txt and instance_parameters.txt)
        _,instance_dir=instance_dir.split('=')
        if instance_dir.startswith('"') and instance_dir.endswith('"'):
            instance_dir = instance_dir[1:-1]
        elif instance_dir.startswith("'") and instance_dir.endswith("'"):
            instance_dir = instance_dir[1:-1]
    else:
        # if not provided, try the default instance directory
        instance_dir=folder_default 

    input_dir=next((p for p in console_input if 'input_dir=' in p),None)
    if input_dir:
        _,input_dir=input_dir.split('=')
        if input_dir.startswith('"') and input_dir.endswith('"'):
            input_dir = input_dir[1:-1]
        elif input_dir.startswith("'") and input_dir.endswith("'"):
            input_dir = input_dir[1:-1]
    else:
        input_dir=instance_dir

    output_dir=next((p for p in console_input if 'output_dir=' in p),None)
    if output_dir:
        _,output_dir=output_dir.split('=')
        if output_dir.startswith('"') and output_dir.endswith('"'):
            output_dir = output_dir[1:-1]
        elif output_dir.startswith("'") and output_dir.endswith("'"):
            output_dir = output_dir[1:-1]
    else:
        output_dir=instance_dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return instance_dir,input_dir,output_dir

def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    locations.set_index('id',inplace=True)

    orders.set_index('order',inplace=True)
    couriers.set_index('courier',inplace=True)
    restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']
    return orders,restaurants,couriers,instanceparams,locations,meters_per_minute,\
           pickup_service_minutes,dropoff_service_minutes,target_click_to_door,\
           pay_per_order,guaranteed_pay_per_hour

def read_solution_information(input_dir):
    # read assignment solution file
    with open(os.path.join(input_dir,'assignment_solution_info.txt'),'r') as f:
        #raw_assignments=[a.replace(' ','\t').replace('\n','').split('\t') for a in f.readlines()]
        raw_assignments=[a.split() for a in f.readlines()]
        assignments=[[int(float(a[0])),int(float(a[1])),a[2],a[3:]] for a in raw_assignments[1:]]
        order_pickup_times=pd.Series(*zip(*[(int(float(a[1])),o) for a in raw_assignments[1:] for o in a[3:]]))
    assignment_sol=pd.DataFrame(data=assignments,columns=['assignment_time','pickup_time','courier','bundle'])

    # read order solution file
    order_sol=pd.read_table(os.path.join(input_dir,'orders_solution_info.txt'),\
                             #names=['order','placement_time','restaurant','latitude','longitude',\
                             #'ready_time','pickup_time','dropoff_time','courier'],\
                             sep=' ')
    order_sol.placement_time=pd.to_numeric(order_sol.placement_time,errors='coerce')
    order_sol.ready_time=pd.to_numeric(order_sol.ready_time,errors='coerce')
    order_sol.pickup_time=pd.to_numeric(order_sol.pickup_time,errors='coerce')
    order_sol.dropoff_time=pd.to_numeric(order_sol.dropoff_time,errors='coerce')
    order_sol.set_index('order',inplace=True)

    # read courier solution file
    with open(os.path.join(input_dir,'courier_solution_info.txt'),'r') as f:
        courier_sol={}
        courier_id=None
        courier_moves=[]
        header=next(f)
        #print(repr(header))
        line=header.split()
        try:
            departure_time=int(float(line[1]))
        except:
            print('detected header line:',header)
            pass
        else: # if the line is not a header
            courier_id=line[0]
            origin_id= line[2] if line[2]!='0' else courier_id
            destination_id=line[3].strip()
            courier_moves.append([departure_time,origin_id,destination_id])        
        for line in f:
            #print(repr(line))
            if line.isspace():
                continue
            else:
                line=line.split()
                if courier_id!=line[0]:
                    courier_sol[courier_id]=courier_moves
                    courier_moves=[]
                    courier_id=line[0]
                departure_time=int(float(line[1]))
                origin_id= line[2] if line[2]!='0' else courier_id
                destination_id=line[3].strip()
                courier_moves.append([departure_time,origin_id,destination_id])
        if courier_id not in courier_sol:
            courier_sol[courier_id]=courier_moves
        courier_sol.pop(None, None)
    return assignment_sol,order_sol,courier_sol,order_pickup_times

# Script
def compute_performance_summary(instance_dir,input_dir,output_dir):
    print('reading instance information')   
    orders,restaurants,couriers,instanceparams,locations,meters_per_minute,\
    pickup_service_minutes,dropoff_service_minutes,target_click_to_door,\
    pay_per_order,guaranteed_pay_per_hour = read_instance_information(instance_dir)
    print('reading solution information')
    assignment_sol,order_sol,courier_sol,order_pickup_times = read_solution_information(input_dir)
    
    ### Check feasibility of solution
    print('checking feasibility of the solution')
    feasibility_file=os.path.join(output_dir,'feasibility_check.txt')
    f= open(feasibility_file, "w")
    feasible=True

    # verify that each order is in at most one assignment
    bundles_per_order={}
    for o in order_sol.index:
        for b in assignment_sol.bundle:
            if o in b:
                if o in bundles_per_order:
                    bundles_per_order[o].append(b)
                else:
                    bundles_per_order[o]=[b]
    violations={o:l for o,l in bundles_per_order.items() if len(l)>1}
    if violations: 
        print('orders in more than one assignment:',file=f)
        print(*violations.keys(),sep='\n',file=f)    
        feasible=False
    else:
        print('every order is in at most one assignment: OK',file=f)

    # verify that assignments are not made before information is revealed
    violations=[]
    orders_per_bundle=[] # leverage loop to record the size of bundles
    for i,a in assignment_sol.iterrows():
        order_seq=a.bundle
        assignment_time=a.assignment_time
        #print('\n'+str(pickup)+' '+str(order_seq),end='\t')
        for o in order_seq:
            placement=orders.at[o,'placement_time']
            #print(ready,end=' ')
            if assignment_time<placement:
                violations.append((assignment_time,placement,o,a))
        orders_per_bundle.append(len(order_seq))
    if violations:
        print('\nassignments made before orders are placed:',file=f)
        print(*violations,sep='\n',file=f)
        feasible=False
    else:
        print('\nassignments are never made before information is revealed: OK',file=f)

    # verify that each assignment is picked up before the off-time of the courier
    violations=[]
    bundles_per_courier={d:0 for d in couriers.index}#leverage loop: couriers' total bundles served
    for i,a in assignment_sol.iterrows():
        offtime=couriers.at[a.courier,'off_time']
        if offtime<a.pickup_time:
            violations.append((offtime,a.pickup_time,a))
        bundles_per_courier[a.courier]+=1
    if violations:
        print('\nbundle picked up after off-time of courier:',file=f)
        print(*violations,sep='\n',file=f)
        feasible=False
    else:
        print('\nbundle picked up before off-time of courier: OK',file=f)

    # verify that, for each assignment, the pickup time is not erlier than the ready time of any order in the bundle
    violations=[]
    for i,a in assignment_sol.iterrows():
        order_seq=a.bundle
        pickup=a.pickup_time
        #print('\n'+str(pickup)+' '+str(order_seq),end='\t')
        for o in order_seq:
            ready=orders.at[o,'ready_time']
            #print(ready,end=' ')
            if ready>pickup:
                violations.append((pickup,ready,a))
    if violations:
        print('\nbundle pickup times do not respect individual ready times:',file=f)
        print(*violations,sep='\n',file=f)
        feasible=False    
    else:
        print('\nbundle pickup times respect ready times: OK',file=f)

    # verify that dropoffs occur in the right order (one assignment after another one, 
    # respecting the delivery sequence in each assigned bundle) and that and delivery 
    # service time is enforced 
    violations=[]
    for i,a in assignment_sol.iterrows():
        order_seq=a.bundle
        dropoffs=[]
        #print('\n',order_seq,end='\t')
        for o in order_seq:
            drop=order_sol.at[o,'dropoff_time']
            #print(drop,end=' ')
            if dropoffs:
                if drop<dropoffs[-1]+dropoff_service_minutes:
                    violations.append((dropoffs,drop,order_seq))
            dropoffs.append(drop)
    if violations:
        print('\ndropoffs do not follow the prescribed sequence:',file=f)
        print(*violations,sep='\n',file=f)
        feasible=False
    else:
        print('\ndropoffs follow the prescribed sequence:OK',file=f)

    # Prepare timeline for each courier: when are they in transit? when and where are
    # they not moving? While we're at it, verify that couriers do not tele-transport 
    # (arrival location is next departure location; arrival happens before departure)
    courier_timeline={}
    violations1=[]
    violations2=[]
    time_driving={}
    for d,s in courier_sol.items():
        courier_timeline[d]=lambda:None
        courier_timeline[d].times=[couriers.loc[d].on_time]
        courier_timeline[d].places=[d]
        time_driving[d]=0
        for a in s:
            if a[1]!=courier_timeline[d].places[-1]:#'current origin should be previous destination'
                violations1.append((d,a[1],courier_timeline[d].places[-1]))
            courier_timeline[d].times.append(a[0])
            courier_timeline[d].places.append('')
            tt=travel_time(a[1],a[2],meters_per_minute,locations)
            courier_timeline[d].times.append(a[0]+tt)
            courier_timeline[d].places.append(a[2])
            time_driving[d]+=tt
        if sorted(courier_timeline[d].times) != courier_timeline[d].times:#'if departures happen after arrivals, times are ordered'
            violations2.append(courier_timeline[d].times)
    if violations1:
        print('\ndiscontinuities in sequence of origin-destination pairs:',file=f)
        print(*violations1,sep='\n',file=f)
        feasible=False
    else:
        print('\ncontinuity in sequence of origin-destination pairs: OK',file=f)
    if violations2:
        print('\ndepartures sometimes happen before arrivals:',file=f)
        print(*violations2,sep='\n',file=f)
        feasible=False
    else:
        print('\ndepartures and arrival time are consistent in time: OK',file=f)

    # Verify that for each dropoff, the courier is located at the right place at the right time
    time_dropping={d:0 for d in couriers.index} #leverage loop: couriers' total dropoff service time
    orders_served={d:0 for d in couriers.index}#leverage loop: couriers' total orders served
    violations=[]
    #print(order_sol.head())
    for o_id, o_info in order_sol.iterrows():
        if o_info.courier=='courier':
            continue
        d=o_info['courier']
        if d not in orders_served:
            continue
        drop=o_info.dropoff_time
        i=bisect.bisect_left(courier_timeline[d].times,drop)-1
        loc_id=courier_timeline[d].places[i]
        if loc_id!=o_id:
            violations.append((o_id,drop,loc_id))
        time_dropping[d]+=dropoff_service_minutes
        orders_served[d]+=1
    if violations:
        print('\ninconsistency in dropoff times and locations',file=f)
        print(*violations,file=f)
        feasible=False
    else:
        print('\ndropoff times and locations are consistent:OK',file=f)

    # Verify that, for each pickup, the courier is located at the right place at the right time
    time_picking={d:0 for d in couriers.index} #leverage loop: couriers' total pickup service time (lower bound)
    violations=[]
    for a_id, a_info in assignment_sol.iterrows():
        d=a_info.courier
        o=a_info.bundle[0]
        r=orders.loc[o].restaurant
        pickup=a_info.pickup_time
        i=bisect.bisect_left(courier_timeline[d].times,pickup)-1
        loc_id=courier_timeline[d].places[i]
        if loc_id!=r:
            violations.append((o_id,r,pickup,loc_id))
        time_picking[d]+=pickup_service_minutes
    if violations:
        print('\ninconsistency in pickup times and locations',file=f)
        print(*violations,file=f)
        feasible=False
    else:
        print('\npickup times and locations are consistent:OK',file=f)

    # Verdict
    if feasible:
        print('FEASIBLE',file=f)
        print('Solution is feasible.')
    else:
        print('INFEASIBLE',file=f)
        print('Solution is not feasible. Check',feasibility_file, 'for more information')
    f.close()
    
    ### Compute performance measures of solution
    print('computing solution performance metrics')
    try:
        #print(order_sol.tail())
        #print(order_sol.dropna().tail())
        #print(len(order_sol))
        #print(len(order_sol.dropna()))
        total_delivered=len(order_sol.dropna())
    except:
        total_delivered=None
    try:
        order_performance=pd.merge(orders.drop(['x','y','restaurant'],axis=1),\
                                   order_sol.drop(['placement_time','ready_time'],axis=1),\
                                   left_index=True,right_index=True)
        #order_performance['pickup_time']=order_pickup_times
        order_performance['click-to-door']=order_performance['dropoff_time']-order_performance['placement_time']
        order_performance['ready-to-door']=order_performance['dropoff_time']-order_performance['ready_time']
        order_performance['ready-to-pickup']=order_performance['pickup_time']-order_performance['ready_time']
        order_performance['click-to-door overage']=order_performance.apply(\
                                                   lambda row:max(0,row['click-to-door']-target_click_to_door) ,axis=1)     
    except:
        order_performance=None
    try:
        courier_performance=couriers.drop(['x','y'],axis=1)
        courier_performance['shift_duration']=courier_performance['off_time']-courier_performance['on_time']
        courier_performance['guaranteed_earnings']=courier_performance['shift_duration']*guaranteed_pay_per_hour/60.0
        courier_performance['orders_delivered']=pd.Series(orders_served)
        courier_performance['bundles_delivered']=pd.Series(bundles_per_courier)
        courier_performance['orders_per_hour']=60*courier_performance['orders_delivered']/courier_performance['shift_duration']
        courier_performance['bundles_per_hour']=60*courier_performance['bundles_delivered']/courier_performance['shift_duration']
        courier_performance['order_earnings']=courier_performance['orders_delivered']*pay_per_order
        courier_performance['payment']=courier_performance.apply(lambda row: max(row['order_earnings'],row['guaranteed_earnings']),axis=1)
        courier_performance['time_driving']=pd.Series(*zip(*[(v,k) for k,v in time_driving.items()]))
        courier_performance['time_dropping']=pd.Series(*zip(*[(v,k) for k,v in time_dropping.items()]))
        courier_performance['time_picking']=pd.Series(*zip(*[(v,k) for k,v in time_picking.items()]))
        courier_performance['utilization']=(courier_performance['time_driving']+courier_performance['time_dropping']+\
                                                 courier_performance['time_picking'])/courier_performance['shift_duration']
        courier_performance.fillna({'utilization':0},inplace=True)
        total_cost=courier_performance['payment'].sum()
        proportion_trueup=len(courier_performance.loc[courier_performance['order_earnings']<courier_performance['guaranteed_earnings']])/(1.0*len(couriers))
    except:
        courier_performance=None
        total_cost=None
        proportion_trueup=None
    try:
        bundle_size=pd.DataFrame(orders_per_bundle,columns=['orders_per_bundle'])
    except:
        bundle_size=None
    if feasible:
        # write performance summary
        performance_file=os.path.join(output_dir,''.join(['solution_performance_',os.getlogin(),'_','f',str(f_minute),'d',str(delta_u),'b',str(beta),'g',str(gamma),'x',str(X),'o',str(omega),'c',str(commitment_strategy),'.txt']))        
        with open(performance_file,'w') as f:
            print('number of orders delivered:',total_delivered,'out of',len(orders),file=f)
            print('total payment:','{0:.2f}'.format(total_cost),file=f)
            print('proportion of couriers receiving minimum guaranteed compensation:',\
                   '{0:.2f}'.format(proportion_trueup),file=f)
            print('\n',file=f)
            print(order_performance[['click-to-door','ready-to-door',\
                            'ready-to-pickup','click-to-door overage']]\
                            .describe(percentiles=[0.1,0.9])\
                            .to_string(float_format=lambda x:'{0:.2f}'.format(x)),file=f)
            print('\n',file=f)
            print(courier_performance[['orders_per_hour','bundles_per_hour','utilization',\
                                       'guaranteed_earnings','order_earnings','payment']]\
                                       .describe(percentiles=[0.1,0.9])\
                                       .to_string(float_format=lambda x:'{0:.2f}'.format(x)),file=f)
            print('\n',file=f)
            print(bundle_size.describe(percentiles=[0.1,0.9])\
                                       .to_string(float_format=lambda x:'{0:.2f}'.format(x)),file=f)
        print('Performance measures were written to file:',performance_file)
    else:
        pass    
    return feasible,total_delivered,total_cost,proportion_trueup,order_performance,courier_performance


instancedir,inputdir,outputdir = instance_dir,instance_dir,instance_dir
print(instance_dir,)
feasible,total_delivered,total_cost,proportion_trueup,order_performance,courier_performance=compute_performance_summary(instancedir,inputdir,outputdir)

./data/5o50t75s1p100
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/5o50t75s1p100/solution_performance_root_f5d10b10g10x25o1000c0.txt


In [15]:
instancedir,inputdir,outputdir = instance_dir,instance_dir,instance_dir
print(instance_dir,)
feasible,total_delivered,total_cost,proportion_trueup,order_performance,courier_performance=compute_performance_summary(instancedir,inputdir,outputdir)

./data/5o50t75s1p100
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/5o50t75s1p100/solution_performance_root_f5d10b10g10x25o1000c0.txt


# Analysis

In [212]:
click_to_door = []
ready_to_door = []
ready_to_pickup = []
order_per_bundle = []
order_per_hour = []
bundle_per_hour = []
undelivered_percent = []
for i in range(10):
    instance_dir = './data/'+str(i)+'o100t100s1p100'

    dr = DeliveryRouting(instance_dir)
    dr.get_ready_orders()
    main_algo(dr)
    feasible,total_delivered,total_cost,proportion_trueup,\
    order_performance,courier_performance=compute_performance_summary(instance_dir,instance_dir,instance_dir)

    click_to_door.append(np.mean(order_performance['click-to-door']))

    ready_to_door.append(np.mean(order_performance['ready-to-door']))

    ready_to_pickup.append(np.mean(order_performance['ready-to-pickup']))
    
    with open(os.path.join(instance_dir,''.join(['solution_performance_',os.getlogin(),'_','f',str(f_minute),'d',str(delta_u),'b',str(beta),'g',str(gamma),'x',str(X),'o',str(omega),'c',str(commitment_strategy),'.txt'])), 'r') as fp:
        # lines to read
        line_numbers = [18,29]
        # get value at the lines
        for i, line in enumerate(fp):
            if i == 0:
                undelivered = float(line.strip().split()[4])/float(line.strip().split()[7]) 
            elif i == 18:
                oph = line.strip().split()[1]
                bph = line.strip().split()[2]
            elif i == 29:
                opb = line.strip().split()[1]
            elif i > 31:
                # don't read after line 31 to save time
                break
    order_per_bundle.append(float(opb))
    order_per_hour.append(float(oph))
    bundle_per_hour.append(float(bph))
    undelivered_percent.append(undelivered)




reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/0o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/1o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/2o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures w

In [247]:
print(click_to_door)
print(ready_to_door)
print(ready_to_pickup) 
print(order_per_bundle)
print(order_per_hour)
print(bundle_per_hour)
print(undelivered_percent)

[33.57085828343313, 34.922764227642276, 32.944365192582026, 33.435331230283914, 37.47606142728094, 39.84594698424895, 37.38101347017319, 36.55061494796594, 33.24016393442623, 34.54696449026346]
[16.451097804391217, 17.900406504065042, 14.838801711840228, 16.138801261829652, 18.355916892502258, 19.63157894736842, 20.61706221937139, 19.147272153894672, 16.772131147540982, 17.75372279495991]
[5.049900199600798, 6.6727642276422765, 3.68188302425107, 4.206098843322818, 6.5501355013550135, 7.944295044179793, 8.995509942270687, 7.159886471144749, 4.874180327868852, 4.893470790378007]
[1.0, 1.0, 1.0, 1.01, 1.0, 1.03, 1.01, 1.0, 1.01, 1.0]
[1.63, 1.94, 1.3, 1.55, 2.19, 2.29, 1.96, 2.22, 1.54, 1.19]
[1.63, 1.93, 1.3, 1.54, 2.18, 2.22, 1.95, 2.21, 1.53, 1.18]
[0.9920792079207921, 0.9144981412639405, 0.9901129943502824, 0.983453981385729, 0.9341772151898734, 0.9555800293685757, 0.9329742669060442, 0.9869281045751634, 0.9983633387888707, 1.0]


In [ ]:
click_to_door = [33.57085828343313, 34.922764227642276, 32.944365192582026, 33.435331230283914, 37.47606142728094, 39.84594698424895, 37.38101347017319, 36.55061494796594, 33.24016393442623, 34.54696449026346]
ready_to_door = [16.451097804391217, 17.900406504065042, 14.838801711840228, 16.138801261829652, 18.355916892502258, 19.63157894736842, 20.61706221937139, 19.147272153894672, 16.772131147540982, 17.75372279495991]
ready_to_pickup = [5.049900199600798, 6.6727642276422765, 3.68188302425107, 4.206098843322818, 6.5501355013550135, 7.944295044179793, 8.995509942270687, 7.159886471144749, 4.874180327868852, 4.893470790378007]
order_per_bundle = [1.0, 1.0, 1.0, 1.01, 1.0, 1.03, 1.01, 1.0, 1.01, 1.0]
order_per_hour = [1.63, 1.94, 1.3, 1.55, 2.19, 2.29, 1.96, 2.22, 1.54, 1.19]
bundle_per_hour = [1.63, 1.93, 1.3, 1.54, 2.18, 2.22, 1.95, 2.21, 1.53, 1.18]
undelivered_percent = [0.9920792079207921, 0.9144981412639405, 0.9901129943502824, 0.983453981385729, 0.9341772151898734, 0.9555800293685757, 0.9329742669060442, 0.9869281045751634, 0.9983633387888707, 1.0]
# build cluster bar chart for click to door, ready to door, ready to pickup
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar([i-0.2 for i in range(10)], click_to_door, width=0.2, label='click to door', align='center')
plt.bar([i for i in range(10)], ready_to_door, width=0.2, label='ready to door', align='center')
plt.bar([i+0.2 for i in range(10)], ready_to_pickup, width=0.2, label='ready to pickup', align='center')
plt.legend()
plt.xlabel('Instance')
plt.ylabel('Time (min)')
plt.title('Time to deliver an order') 
# change xticks to 10 instances
plt.xticks(range(10))
# add lable to each bar
for a,b in zip(range(10),[round(i,0) for i in click_to_door]):
    plt.text(a-0.2, b-1.8, "{:.0f}".format(b),horizontalalignment='center')
for a,b in zip(range(10),[round(i,0) for i in ready_to_door]):
    plt.text(a, b+1.2, "{:.0f}".format(b),horizontalalignment='center')
for a,b in zip(range(10),[round(i,0) for i in ready_to_pickup]):
    plt.text(a+0.2, b+1.5, "{:.0f}".format(b),horizontalalignment='center')
# save figure
plt.savefig(r'C:\Users\Asus\OneDrive\Desktop\time_to_deliver.png')


In [ ]:
# build cluster bar chart for order per bundle, order per hour, bundle per hour
plt.figure(figsize=(10, 6))
plt.bar([i-0.2 for i in range(10)], order_per_bundle, width=0.2, label='order per bundle', align='center')
plt.bar([i for i in range(10)], order_per_hour, width=0.2, label='order per hour', align='center')
plt.bar([i+0.2 for i in range(10)], bundle_per_hour, width=0.2, label='bundle per hour', align='center')
plt.legend()
plt.xlabel('Instance')
plt.ylabel('Number')
plt.title('Number of orders and bundles')
# change xticks to 10 instances
plt.xticks(range(10))
# add lable to each bar
for a,b in zip(range(10), order_per_bundle):
    plt.text(a-0.2, b+0.05, str(b),horizontalalignment='center')
for a,b in zip(range(10),order_per_hour):
    plt.text(a, b+0.05, str(b),horizontalalignment='center')
for a,b in zip(range(10), bundle_per_hour):
    plt.text(a+0.2, b-0.1, str(b),horizontalalignment='center')
# save figure
plt.savefig(r'C:\Users\Asus\OneDrive\Desktop\order_bundle.png')

In [ ]:
# build stacked bar chart for undelivered percent
import matplotlib.ticker as mtick
plt.figure(figsize=(10, 6))
plt.bar([i for i in range(10)], [round(i,2) for i in undelivered_percent], width=0.8, label='Delivered percent', color='green',align='center')
plt.bar([i for i in range(10)], [1-round(i,2) for i in undelivered_percent],width=0.8, bottom=[round(i,2) for i in undelivered_percent],label='Undelivered percent', color='orange', align='center')
plt.legend()
plt.xlabel('Instance')
plt.ylabel('Percent')
plt.title('Percentage of Delivered Orders ')
plt.ylim([0,1])
# set y axis lable format to percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{}%'.format))
# change xticks to 10 instances
plt.xticks(range(10))
# add lable to each bar
for a,b in zip(range(10), [round(i,3) for i in undelivered_percent]):
    plt.text(a, b-0.5, '{}%'.format(str(round(b*100,3))),horizontalalignment='center')
# save figure
plt.savefig(r'C:\Users\Asus\OneDrive\Desktop\undelivered.png')

**balance between click to door time and delivered percent**

In [ ]:
# run script
mean_click_to_door = []
mean_delivered_percent = []
# for om in [0]:
for om in [1,10,50,100,500,1000]:
    # Accorting to default values in the paper
    f_minute = 5
    delta_u = 10
    # Not find in the paper
    beta = 10 # should be tuned
    gamma = 10 # should be tuned
    X = 25 # max pickup delay for commitment strategy, should be tuned
    omega = om # control undelivered orders
    commitment_strategy = 0 # 0: no commitment, 1: commitment

    click_to_door = []
    delivered_percent = []
    for j in range(10):
        instance_dir = './data/'+str(j)+'o100t100s1p100'

        dr = DeliveryRouting(instance_dir)
        dr.get_ready_orders()
        main_algo(dr)
        feasible,total_delivered,total_cost,proportion_trueup,\
        order_performance,courier_performance=compute_performance_summary(instance_dir,instance_dir,instance_dir)

        click_to_door.append(np.mean(order_performance['click-to-door']))

        with open(os.path.join(instance_dir,''.join(['solution_performance_',os.getlogin(),'_','f',str(f_minute),'d',str(delta_u),'b',str(beta),'g',str(gamma),'x',str(X),'o',str(omega),'c',str(commitment_strategy),'.txt'])), 'r') as fp:
            # lines to read
            line_numbers = [18,29]
            # get value at the lines
            for i, line in enumerate(fp):
                if i == 0:
                    delivered = float(line.strip().split()[4])/float(line.strip().split()[7]) 
                elif i == 18:
                    oph = line.strip().split()[1]
                    bph = line.strip().split()[2]
                elif i == 29:
                    opb = line.strip().split()[1]
                elif i > 31:
                    # don't read after line 31 to save time
                    break
        delivered_percent.append(delivered)
    
    mean_click_to_door.append(np.mean(click_to_door))
    mean_delivered_percent.append(np.mean(delivered_percent))


In [ ]:
print(mean_click_to_door)
print(mean_delivered_percent)

# build line plot for click to door time  and delivered percent
plt.figure(figsize=(10, 6))
plt.plot(mean_click_to_door, [round(i*100,2) for i in mean_delivered_percent], color='royalblue', marker='o')
plt.legend()
plt.xlabel('Click to door time (min)')
plt.ylabel('Delivered order (%)')
plt.title('Click to door time Vs Delivered Order Percentage')
plt.ylim([0,100])
plt.xlim([32,35.5])
# set y axis lable format to percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:.0f}%'.format))





# add lable to each point
# for a,b in zip(mean_click_to_door, mean_delivered_percent):
#     plt.text(a, b, '({},{})'.format(round(a,3),round(b,3)),horizontalalignment='center')
# save figure
plt.savefig(r'C:\Users\Asus\OneDrive\Desktop\balance_ctd_do.png')

**changing delta u**

In [187]:
# run script changing delta_u
mean_click_to_door = []

# for om in [0]:
for delta in range(10,11):
    # Accorting to default values in the paper
    f_minute = 5
    delta_u = delta
    # Not find in the paper
    beta = 10 # should be tuned
    gamma = 10 # should be tuned
    X = 25 # max pickup delay for commitment strategy, should be tuned
    omega = 1000 # control undelivered orders
    commitment_strategy = 0 # 0: no commitment, 1: commitment

    click_to_door = []

    for j in range(10):
        instance_dir = './data/'+str(j)+'o100t100s1p100'

        dr = DeliveryRouting(instance_dir)
        dr.get_ready_orders()
        main_algo(dr)
        feasible,total_delivered,total_cost,proportion_trueup,\
        order_performance,courier_performance=compute_performance_summary(instance_dir,instance_dir,instance_dir)

        click_to_door.append(np.mean(order_performance['click-to-door']))

    mean_click_to_door.append(np.mean(click_to_door))



reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/0o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/1o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/2o100t100s1p100\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures w

In [ ]:
# result mean_click_to_door
# [37.72774695166477,
#  37.24947317586074,
#  36.75132811691289,
#  36.22726689204738,
#  35.779687564295735,
#  35.391408418830004,
#  35.289124522234104,
#  35.110925636612,
#  34.83862550050549,
#  34.665620894324356,
#  34.62896379702127,
#  34.6362809710615,
#  34.58265758062581,
#  34.52002777927102,
#  34.46688378591767,
#  34.51358721669067,
#  34.51099803912221,
#  34.50061176332542,
#  34.73228584430735,
#  34.63070285138495,
#  34.67482867486875,
#  34.85394522345103,
#  34.77682636426921,
#  34.937884949060006,
#  35.03567947151257,
#  34.993035255395355]
len(mean_click_to_door)

# build line plot for mean click to door
plt.figure(figsize=(10, 6))
plt.plot(range(5,31), mean_click_to_door, color='royalblue', marker='o')
plt.legend()
plt.xlabel('Delta_u')
plt.ylabel('Mean click to door time (min)')
plt.title('Mean click to door time by horizon time')
plt.ylim([32,40])
plt.xlim([4,31])
# save figure
plt.savefig(r'C:\Users\Asus\OneDrive\Desktop\delta_u_ctd.png')

**Effect of commitment strategy**

In [ ]:
f_minute = 5
delta_u = delta
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned
X = 25 # max pickup delay for commitment strategy, should be tuned
omega = 1000 # control undelivered orders
commitment_strategy = 1 # 0: no commitment, 1: commitment

click_to_door = []
ready_to_door = []
ready_to_pickup = []
order_per_bundle = []
order_per_hour = []
bundle_per_hour = []
undelivered_percent = []
for i in range(10):
    instance_dir = './data/'+str(i)+'o100t100s1p100'

    dr = DeliveryRouting(instance_dir)
    dr.get_ready_orders()
    main_algo(dr)
    feasible,total_delivered,total_cost,proportion_trueup,\
    order_performance,courier_performance=compute_performance_summary(instance_dir,instance_dir,instance_dir)

    click_to_door.append(np.mean(order_performance['click-to-door']))

    ready_to_door.append(np.mean(order_performance['ready-to-door']))

    ready_to_pickup.append(np.mean(order_performance['ready-to-pickup']))
    
    with open(os.path.join(instance_dir,''.join(['solution_performance_',os.getlogin(),'_','f',str(f_minute),'d',str(delta_u),'b',str(beta),'g',str(gamma),'x',str(X),'o',str(omega),'c',str(commitment_strategy),'.txt'])), 'r') as fp:
        # lines to read
        line_numbers = [18,29]
        # get value at the lines
        for i, line in enumerate(fp):
            if i == 0:
                undelivered = float(line.strip().split()[4])/float(line.strip().split()[7]) 
            elif i == 18:
                oph = line.strip().split()[1]
                bph = line.strip().split()[2]
            elif i == 29:
                opb = line.strip().split()[1]
            elif i > 31:
                # don't read after line 31 to save time
                break
    order_per_bundle.append(float(opb))
    order_per_hour.append(float(oph))
    bundle_per_hour.append(float(bph))
    undelivered_percent.append(undelivered)


In [201]:
print(np.mean(click_to_door))
print(np.mean(ready_to_door))
print(np.mean(ready_to_pickup))
print(np.mean(order_per_bundle))
print(np.mean(order_per_hour))
print(np.mean(bundle_per_hour))
print(np.mean(undelivered_percent))

42.423347847810156
24.70357893631825
9.928713635410459
1.349
1.714
1.2710000000000001
0.9350191722099715


**Uncontrol factor**

In [204]:
f_minute = 5
delta_u = delta
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned
X = 25 # max pickup delay for commitment strategy, should be tuned
omega = 1000 # control undelivered orders
commitment_strategy = 0 # 0: no commitment, 1: commitment

click_to_door = []
ready_to_door = []
ready_to_pickup = []
order_per_bundle = []
order_per_hour = []
bundle_per_hour = []
undelivered_percent = []
for i in range(10):
    instance_dir = './data/'+str(i)+'o100t100s1p125'

    dr = DeliveryRouting(instance_dir)
    dr.get_ready_orders()
    main_algo(dr)
    feasible,total_delivered,total_cost,proportion_trueup,\
    order_performance,courier_performance=compute_performance_summary(instance_dir,instance_dir,instance_dir)

    click_to_door.append(np.mean(order_performance['click-to-door']))

    ready_to_door.append(np.mean(order_performance['ready-to-door']))

    ready_to_pickup.append(np.mean(order_performance['ready-to-pickup']))
    
    with open(os.path.join(instance_dir,''.join(['solution_performance_',os.getlogin(),'_','f',str(f_minute),'d',str(delta_u),'b',str(beta),'g',str(gamma),'x',str(X),'o',str(omega),'c',str(commitment_strategy),'.txt'])), 'r') as fp:
        # lines to read
        line_numbers = [18,29]
        # get value at the lines
        for i, line in enumerate(fp):
            if i == 0:
                undelivered = float(line.strip().split()[4])/float(line.strip().split()[7]) 
            elif i == 18:
                oph = line.strip().split()[1]
                bph = line.strip().split()[2]
            elif i == 29:
                opb = line.strip().split()[1]
            elif i > 31:
                # don't read after line 31 to save time
                break
    order_per_bundle.append(float(opb))
    order_per_hour.append(float(oph))
    bundle_per_hour.append(float(bph))
    undelivered_percent.append(undelivered)


reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/0o100t100s1p125\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/1o100t100s1p125\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures were written to file: ./data/2o100t100s1p125\solution_performance_Asus_f5d10b10g10x25o1000c0.txt
reading instance information
reading solution information
checking feasibility of the solution
Solution is feasible.
computing solution performance metrics
Performance measures w

In [205]:
print(np.mean(click_to_door))
print(np.mean(ready_to_door))
print(np.mean(ready_to_pickup))
print(np.mean(order_per_bundle))
print(np.mean(order_per_hour))
print(np.mean(bundle_per_hour))
print(np.mean(undelivered_percent))

39.242721326119195
17.623143497311453
5.880480857248163
1.003
1.777
1.768
0.9657658127456786


In [250]:
# Measure the improvement of Local search
dr = DeliveryRouting(instance_dir)
dr.get_ready_orders()

# time horizon: running the algorithm every f minutes
t_list = [*range(0, 24*60+1, f_minute)]

before = 0
after = 0
for t in t_list:
    ready_orders = dr.get_ready_orders_at_t(t)
    # get idle couriers
    idle_couriers = dr.get_idle_courier_at_t(t)
    bundle_size = int(dr.get_bundle_size(t))
    # if there are orders to be delivered, run the algorithm
    if len(ready_orders)>0:
        list_of_routes_by_restaurant = dr.initialization(t,ready_orders,idle_couriers,bundle_size)
        for res in list_of_routes_by_restaurant:
            for rout in res:
                before += rout.get_route_cost(dr.meters_per_minute,dr.locations)
        list_of_routes_by_restaurant = dr.local_search(list_of_routes_by_restaurant)
        for res in list_of_routes_by_restaurant:
            for rout in res:
                after += rout.get_route_cost(dr.meters_per_minute,dr.locations)
print(before, after)


617363.0 617350.0
